In [1]:
import pandas as pd
import geopandas as gp
import folium
import numpy as np 

%matplotlib inline

In [2]:
com_area = gp.read_file('D:/TNC-Demand-Model/Inputs/Chicago Community Areas/geo_export_d8da94d2-4ef1-4ee6-9fff-7bb20d451fe2.shp')

In [3]:
com_area.area_num_1 = com_area.area_num_1.astype(float)

In [26]:
years = [2018,2019,2020]
tods = [1,2,3,4,5]
df3 = pd.DataFrame()
df4 =  pd.DataFrame()
for year in years:
    print('Working on year ' + str(year))
    
    if year == 2018:
        months = [11,12]
    elif year == 2020:
        months = [1,2]
    elif year == 2019:
        months = [1,2,3,4,5,6,7,8,9,10,11,12]
    else:
        print('Bad Year!')
        
    for month in months:

        print('Working on month ' + str(month))
    
        for tod in tods:
            df = pd.read_hdf('D:/TNC-Demand-Model/Inputs/Chicago Ride-Hailing/Chicago_TNC_Trips_20.H5',where = ['YEAR == ' + str(year),'MONTH == '+str(month)], key = 'Weekday_' + str(tod))
            df2 = df[(~np.isnan(df['Dropoff Community Area']))&(~np.isnan(df['Pickup Community Area']))]
            df2 = df2[(np.isnan(df2['Dropoff Census Tract']))|(np.isnan(df2['Pickup Census Tract']))]
            df2['SUPPRESSED_TRIPS'] = 1
            df2['DAY'] = df2['Trip Start Timestamp'].dt.day

            
            df_unsup = df[(~np.isnan(df['Dropoff Community Area']))&(~np.isnan(df['Pickup Community Area']))]
            df_unsup = df_unsup[(~np.isnan(df['Dropoff Census Tract']))&(~np.isnan(df_unsup['Pickup Census Tract']))]
            df_unsup['UNSUPPRESSED_TRIPS'] = 1
            df_unsup['DAY'] = df_unsup['Trip Start Timestamp'].dt.day
            
            df3 = df3.append(df2)
            df4 = df4.append(df_unsup)
            
        sup_pickup_daily = df3.groupby(by = ['MONTH','YEAR','DAY','Pickup Community Area'],as_index = False).sum()
        sup_pickup_monthly_sum = sup_pickup_daily.groupby(by = ['MONTH','YEAR','Pickup Community Area'],as_index = False).sum()
        sup_pickup_monthly_avg = sup_pickup_daily.groupby(by = ['MONTH','YEAR','Pickup Community Area'],as_index = False).mean()

        sup_dropoff_daily = df3.groupby(by = ['MONTH','YEAR','DAY','Dropoff Community Area'],as_index = False).sum()
        sup_dropoff_monthly_sum = sup_dropoff_daily.groupby(by = ['MONTH','YEAR','Dropoff Community Area'],as_index = False).sum()
        sup_dropoff_monthly_avg = sup_dropoff_daily.groupby(by = ['MONTH','YEAR','Dropoff Community Area'],as_index = False).mean()
       
        com_area.area_num_1 = com_area.area_num_1.astype(float)
        
        
        #run this block of code for the share of trips suppressed maps... make sure you change the inputs of the folium map section
        
        #unsup_pickup_monthly = df4.groupby(by = ['MONTH','YEAR','Pickup Community Area'],as_index = False).sum()
        #unsup_dropoff_monthly = df4.groupby(by = ['MONTH','YEAR','Dropoff Community Area'],as_index = False).sum()

        
       # join_pickup = sup_pickup_monthly_sum.merge(unsup_pickup_monthly, on = ['MONTH','YEAR','Pickup Community Area'])
        #join_dropoff = sup_dropoff_monthly_sum.merge(unsup_dropoff_monthly, on = ['MONTH','YEAR','Dropoff Community Area'])

        #join_pickup['Share of Trips Suppressed'] = join_pickup['SUPPRESSED_TRIPS']/(join_pickup['UNSUPPRESSED_TRIPS'] + join_pickup['SUPPRESSED_TRIPS'])
        #join_dropoff['Share of Trips Suppressed'] = join_dropoff['SUPPRESSED_TRIPS']/(join_dropoff['UNSUPPRESSED_TRIPS'] + join_dropoff['SUPPRESSED_TRIPS'])

    
        m = folium.Map([41.8781, -87.6298], zoom_start=11)

        # Add the color for the chloropleth:
        folium.Choropleth(
         geo_data=com_area,
         name= "Share of Pickups Suppressed",
         data=sup_pickup_monthly_avg,
         columns = ['Pickup Community Area', 'SUPPRESSED_TRIPS'],
         key_on='feature.properties.area_num_1',
         fill_color='BuGn',
         fill_opacity=0.6,
         line_opacity=0.2,
         legend_name='Average Weekday Pickups',
         highlight = True
        ).add_to(m)

        folium.Choropleth(
         geo_data=com_area,
         name= "Average Weekday Suppressed Trips",
         data=sup_dropoff_monthly_avg,
         columns = ['Dropoff Community Area','SUPPRESSED_TRIPS'],
         key_on='feature.properties.area_num_1',
         fill_color='BuGn',
         fill_opacity=0.6,
         line_opacity=0.2,
         legend_name='Average Weekday Dropoffs',
         highlight = True
        ).add_to(m)

        #feature_group = folium.FeatureGroup(name='Census Tract Centroids', show = False)


        #for tract2 in centroids.GEOID:
        #    row = centroids[centroids['GEOID'] == tract2]
        #    folium.CircleMarker([row['Y'], row['X']], popup = str(int(row['GEOID'].values[0])), radius = 1, fill = True, fill_color = 'grey', color = 'grey').add_to(feature_group)

        #m.add_child(feature_group)


        folium.LayerControl().add_to(m)

        m.save('D:/TNC-Demand-Model/Data Exploration/Suppressed Ridehailing Maps/' + str(year) +'_' + str(month) + '_Suppressed_Trips.html')

print('Everything is Complete!')

Working on year 2018
Working on month 11


C:\Users\transportlab\anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Working on month 12
Working on year 2019
Working on month 1
Working on month 2
Working on month 3
Working on month 4
Working on month 5
Working on month 6
Working on month 7
Working on month 8
Working on month 9
Working on month 10
Working on month 11
Working on month 12
Working on year 2020
Working on month 1
Working on month 2
Everything is Complete!


In [27]:
sup_monthly_daily = df3.groupby(by = ['MONTH','YEAR','DAY'],as_index = False).sum()
sup_monthly = sup_monthly_daily[['MONTH','YEAR','SUPPRESSED_TRIPS']].groupby(['YEAR','MONTH'],as_index = False).mean()



unsup_monthly_daily = df4.groupby(by = ['MONTH','YEAR','DAY'],as_index = False).sum()
unsup_monthly = unsup_monthly_daily[['MONTH','YEAR','UNSUPPRESSED_TRIPS']].groupby(['YEAR','MONTH'],as_index = False).mean()

In [46]:
unsup_hourly_daily = df4.groupby(by = ['DAY','HOUR'],as_index = False).sum()
unsup_hourly = unsup_hourly_daily[['HOUR','UNSUPPRESSED_TRIPS']].groupby(by = ['HOUR']).mean()


sup_hourly_daily = df3.groupby(by = ['DAY','HOUR'],as_index = False).sum()
sup_hourly = sup_hourly_daily[['HOUR','SUPPRESSED_TRIPS']].groupby(by = ['HOUR']).mean()

In [1]:
sup_monthly

NameError: name 'sup_monthly' is not defined

In [29]:
joined = pd.DataFrame()

In [30]:
joined['SUPPRESSED_TRIPS'] = sup_monthly['SUPPRESSED_TRIPS']

In [31]:
joined['UNSUPPRESSED_TRIPS'] = unsup_monthly['UNSUPPRESSED_TRIPS']

In [32]:
joined.index = sup_monthly.index

In [33]:
joined['Share of Trips Suppressed'] = joined['SUPPRESSED_TRIPS']/(joined['UNSUPPRESSED_TRIPS']+joined['SUPPRESSED_TRIPS'])

In [34]:
joined

,SUPPRESSED_TRIPS,UNSUPPRESSED_TRIPS,Share of Trips Suppressed
0,55854.909091,185028.954545,0.231875
1,55071.761905,167653.904762,0.247263
2,56019.782609,177809.043478,0.239576
3,57076.100000,204452.300000,0.218241
4,58181.238095,212138.476190,0.215231
5,58273.681818,193462.136364,0.231487
6,59239.826087,193690.521739,0.234214
7,58941.650000,194021.050000,0.233005
8,57197.260870,177963.608696,0.243226
9,57047.090909,183699.181818,0.236959


In [62]:
joined = pd.DataFrame()

In [63]:
joined['UNSUPPRESSED_TRIPS'] = unsup_hourly['UNSUPPRESSED_TRIPS']

In [64]:
joined['SUPPRESSED_TRIPS'] = sup_hourly['SUPPRESSED_TRIPS']

In [57]:
joined.index = sup_hourly.index

In [65]:
joined['Share of Trips Suppressed'] = joined['SUPPRESSED_TRIPS']/(joined['UNSUPPRESSED_TRIPS']+joined['SUPPRESSED_TRIPS'])

In [66]:
joined

,UNSUPPRESSED_TRIPS,SUPPRESSED_TRIPS,Share of Trips Suppressed
HOUR,,,
1,17580.483871,21493.354839,0.550070
2,8857.290323,17473.516129,0.663615
3,4365.774194,14766.774194,0.771814
4,7306.000000,17619.903226,0.706891
5,18383.354839,22429.129032,0.549565
6,47749.935484,27990.806452,0.369561
7,103158.580645,30891.161290,0.230446
8,143148.258065,30283.290323,0.174612
9,119232.774194,30530.322581,0.203857
